# Проект: Принятие решений в бизнесе

### Задача

Отдел Маркетинга крупного интернет-магазина подготовил список гипотез для увеличения выручки, которые требуется проверить Аналитикам.

#### Этапы проекта:

- приоритизировать гипотезы,
- запустить A/B-тест
- проанализировать результаты. 

#### Предоставлены данные и их описание:

##### Файл: /datasets/hypothesis.csv
- Hypothesis — краткое описание гипотезы;
- Reach — охват пользователей по 10-балльной шкале;
- Impact — влияние на пользователей по 10-балльной шкале;
- Confidence — уверенность в гипотезе по 10-балльной шкале;
- Efforts — затраты ресурсов на проверку гипотезы по 10-балльной шкале. Чем больше значение Efforts, тем дороже проверка гипотезы.

##### Файл: /datasets/orders.csv
- transactionId — идентификатор заказа;
- visitorId — идентификатор пользователя, совершившего заказ;
- date — дата, когда был совершён заказ;
- revenue — выручка заказа;
- group — группа A/B-теста, в которую попал заказ.

##### Файл: /datasets/visitors.csv
- date — дата;
- group — группа A/B-теста;
- visitors — количество пользователей в указанную дату в указанной группе A/B-теста

In [167]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import datetime as dt
import seaborn as sns
from scipy import stats as stp

pd.set_option('display.max_columns', None)
pd.options.display.max_colwidth = 200

In [168]:
#скачаем датасеты с: гипотезами, заказами и посещениями

try:
    hypothesis  = pd.read_csv('C:/Users/rb182797/hypothesis.csv')
    orders = pd.read_csv('C:/Users/rb182797/orders.csv')
    visitors = pd.read_csv('C:/Users/rb182797/visitors.csv')
    #hypothesis  = pd.read_csv('/datasets/hypothesis.csv')
    #orders = pd.read_csv('/datasets/orders.csv')
    #visitors = pd.read_csv('/datasets/visitors.csv')
except:
    hypothesis  = pd.read_csv('https://code.s3.yandex.net/datasets/hypothesis.csv')
    orders = pd.read_csv('https://code.s3.yandex.net/datasets/orders.csv')
    visitors = pd.read_csv('https://code.s3.yandex.net/datasets/visitors.csv')

### Предобработка и подготовка данных
#### Таблица hypothesis

In [169]:
hypothesis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Hypothesis  9 non-null      object
 1   Reach       9 non-null      int64 
 2   Impact      9 non-null      int64 
 3   Confidence  9 non-null      int64 
 4   Efforts     9 non-null      int64 
dtypes: int64(4), object(1)
memory usage: 492.0+ bytes


In [170]:
display(hypothesis)

,Hypothesis,Reach,Impact,Confidence,Efforts
0,"Добавить два новых канала привлечения трафика, что позволит привлекать на 30% больше пользователей",3,10,8,6
1,"Запустить собственную службу доставки, что сократит срок доставки заказов",2,5,4,10
2,"Добавить блоки рекомендаций товаров на сайт интернет магазина, чтобы повысить конверсию и средний чек заказа",8,3,7,3
3,"Изменить структура категорий, что увеличит конверсию, т.к. пользователи быстрее найдут нужный товар",8,3,3,8
4,"Изменить цвет фона главной страницы, чтобы увеличить вовлеченность пользователей",3,1,1,1
5,"Добавить страницу отзывов клиентов о магазине, что позволит увеличить количество заказов",3,2,2,3
6,"Показать на главной странице баннеры с актуальными акциями и распродажами, чтобы увеличить конверсию",5,3,8,3
7,"Добавить форму подписки на все основные страницы, чтобы собрать базу клиентов для email-рассылок",10,7,8,5
8,"Запустить акцию, дающую скидку на товар в день рождения",1,9,9,5


#### Таблица orders

In [171]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1197 entries, 0 to 1196
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   transactionId  1197 non-null   int64 
 1   visitorId      1197 non-null   int64 
 2   date           1197 non-null   object
 3   revenue        1197 non-null   int64 
 4   group          1197 non-null   object
dtypes: int64(3), object(2)
memory usage: 46.9+ KB


In [172]:
orders.head()

,transactionId,visitorId,date,revenue,group
0,3667963787,3312258926,2019-08-15,1650,B
1,2804400009,3642806036,2019-08-15,730,B
2,2961555356,4069496402,2019-08-15,400,A
3,3797467345,1196621759,2019-08-15,9759,B
4,2282983706,2322279887,2019-08-15,2308,B


In [173]:
orders[orders.duplicated()].count()

transactionId    0
visitorId        0
date             0
revenue          0
group            0
dtype: int64

In [174]:
orders['count_numbers'] = len(str(orders['transactionId']))
orders['count_numbers'].value_counts()

count_numbers
256    1197
Name: count, dtype: int64

In [175]:
orders['count_numbers'] = len(str(['visitorId']))
orders['count_numbers'].value_counts()

count_numbers
13    1197
Name: count, dtype: int64

In [176]:
print(orders['revenue'].min())
print(orders['revenue'].max())

50
1294500


In [177]:
orders['date'] = pd.to_datetime(orders['date'])
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1197 entries, 0 to 1196
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   transactionId  1197 non-null   int64         
 1   visitorId      1197 non-null   int64         
 2   date           1197 non-null   datetime64[ns]
 3   revenue        1197 non-null   int64         
 4   group          1197 non-null   object        
 5   count_numbers  1197 non-null   int64         
dtypes: datetime64[ns](1), int64(4), object(1)
memory usage: 56.2+ KB


In [178]:
print(orders['date'].min())
print(orders['date'].max())

2019-08-01 00:00:00
2019-08-31 00:00:00


In [179]:
orders['group'].value_counts()

group
B    640
A    557
Name: count, dtype: int64

#### Таблица visitors

In [180]:
visitors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   date      62 non-null     object
 1   group     62 non-null     object
 2   visitors  62 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 1.6+ KB


In [181]:
visitors[visitors.duplicated()].count()

date        0
group       0
visitors    0
dtype: int64

In [182]:
visitors.head()

,date,group,visitors
0,2019-08-01,A,719
1,2019-08-02,A,619
2,2019-08-03,A,507
3,2019-08-04,A,717
4,2019-08-05,A,756


In [183]:
visitors['date'] = pd.to_datetime(visitors['date'])
visitors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      62 non-null     datetime64[ns]
 1   group     62 non-null     object        
 2   visitors  62 non-null     int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 1.6+ KB


In [184]:
print(visitors['date'].min())
print(visitors['date'].max())

2019-08-01 00:00:00
2019-08-31 00:00:00


In [185]:
visitors['group'].value_counts()

group
A    31
B    31
Name: count, dtype: int64

In [186]:
print(visitors['visitors'].min())
print(visitors['visitors'].max())

361
770


#### Вывод по итогам преобработки данных

##### Таблица hypothesis

Таблца включает 9 гипотез с оценками от 1 до 10 по праметрам Reach, Impact, Confidence, Efforts.

##### Таблица orders

Таблица содержит 1 197 (группы A=557, группа В=640) строк, в которых столбцы: ID заказа, ID пользователя, который совершил заказ, дату заказа, выручку (50-1 294 500) и группа A/B-теста, в которую попал заказ. Дубликатов и пропусков не обнаружено. Перевели дату в формат datetime, период с 2019-08-01 по 2019-08-31.

##### Таблица visitors

Таблица содержит 62 строки, в которых: дата, группа A/B-теста, количество пользователей в указанную дату в указанной группе A/B-теста (361-770) . Дубликатов и пропусков не обнаружено. Перевели дату в формат datetime, период с 2019-08-01 по 2019-08-31.

###### Аномалий не обнаружено.

## Часть 1. Приоритизация гипотез.

#### Задача

- Применить фреймворк **ICE**  для приоритизации гипотез. Отсортируйте их по убыванию приоритета.
- Применить фреймворк **RICE** для приоритизации гипотез. Отсортируйте их по убыванию приоритета.
- Указать, как изменилась приоритизация гипотез при применении **RICE** вместо **ICE**. Объяснить, почему так произошло.


#### Фреймворк ICE

## Часть 2. Анализ A/B-теста

#### Задача

Проанализируйте A/B-тест:

#### 1. Постройте график кумулятивной выручки по группам. Сделайте выводы и предположения.
#### 2. Постройте график кумулятивного среднего чека по группам. Сделайте выводы и предположения.
#### 3. Постройте график относительного изменения кумулятивного среднего чека группы B к группе A. Сделайте выводы и предположения.
#### 4. Постройте график кумулятивного среднего количества заказов на посетителя по группам. Сделайте выводы и предположения.
#### 5. Постройте график относительного изменения кумулятивного среднего количества заказов на посетителя группы B к группе A. Сделайте выводы и предположения.
#### 6. Постройте точечный график количества заказов по пользователям. Сделайте выводы и предположения.
#### 7. Посчитайте 95-й и 99-й перцентили количества заказов на пользователя. Выберите границу для определения аномальных пользователей.
#### 8. Постройте точечный график стоимостей заказов. Сделайте выводы и предположения.
#### 9. Посчитайте 95-й и 99-й перцентили стоимости заказов. Выберите границу для определения аномальных заказов.
#### 10. Посчитайте статистическую значимость различий в среднем количестве заказов на посетителя между группами по «сырым» данным. Сделайте выводы и предположения.
#### 11. Посчитайте статистическую значимость различий в среднем чеке заказа между группами по «сырым» данным. Сделайте выводы и предположения.
#### 12. Посчитайте статистическую значимость различий в среднем количестве заказов на посетителя между группами по «очищенным» данным. Сделайте выводы и предположения.
#### 13. Посчитайте статистическую значимость различий в среднем чеке заказа между группами по «очищенным» данным. Сделайте выводы и предположения.
#### 14. Примите решение по результатам теста и объясните его. Варианты решений:
- Остановить тест, зафиксировать победу одной из групп.
- Остановить тест, зафиксировать отсутствие различий между группами.
- Продолжить тест.

In [187]:
display(hypothesis)

,Hypothesis,Reach,Impact,Confidence,Efforts
0,"Добавить два новых канала привлечения трафика, что позволит привлекать на 30% больше пользователей",3,10,8,6
1,"Запустить собственную службу доставки, что сократит срок доставки заказов",2,5,4,10
2,"Добавить блоки рекомендаций товаров на сайт интернет магазина, чтобы повысить конверсию и средний чек заказа",8,3,7,3
3,"Изменить структура категорий, что увеличит конверсию, т.к. пользователи быстрее найдут нужный товар",8,3,3,8
4,"Изменить цвет фона главной страницы, чтобы увеличить вовлеченность пользователей",3,1,1,1
5,"Добавить страницу отзывов клиентов о магазине, что позволит увеличить количество заказов",3,2,2,3
6,"Показать на главной странице баннеры с актуальными акциями и распродажами, чтобы увеличить конверсию",5,3,8,3
7,"Добавить форму подписки на все основные страницы, чтобы собрать базу клиентов для email-рассылок",10,7,8,5
8,"Запустить акцию, дающую скидку на товар в день рождения",1,9,9,5
